### 0. Imports

In [1]:
import pandas as pd
import numpy as np

### 1. Load and Prepare Data

#### 1a) Demand Data

In [2]:
demand = pd.read_csv('../data/full_pge_demand.csv')

In [3]:
# convert datetime string to period object
demand['utc'] = [pd.Period(period, freq='H') for period in demand['utc']]

In [4]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63298 entries, 0 to 63297
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype    
---  ------      --------------  -----    
 0   utc         63298 non-null  period[H]
 1   demand_MWh  63298 non-null  float64  
dtypes: float64(1), period[H](1)
memory usage: 989.2 KB


#### 1b) Weather Data

In [5]:
weather = pd.read_csv('../data/PDX_weather_daily.csv')

In [6]:
weather.head(12)

,Daily Temperature and Precipitation Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,"Portland, Oregon Airport",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Period of Record,NaN,NaN,October 1940 to April 2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,File last updated:,NaN,NaN,NaN,25-May-22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,"TX is Maximum Temperature (deg F), TN is Minim...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Example: High Temperature 23 October 1940 i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,YR,MO,NaN,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
9,1940,10,TX,M,M,M,M,M,M,M,...,58,57,57,57,56,53,59,59,52,M


The first 8 rows of the CSV are not useful.

In [7]:
weather = weather[8:]

In [8]:
weather.head()

,Daily Temperature and Precipitation Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
8,YR,MO,NaN,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
9,1940,10,TX,M,M,M,M,M,M,M,...,58,57,57,57,56,53,59,59,52,M
10,1940,10,TN,M,M,M,M,M,M,M,...,53,48,41,38,37,45,48,50,46,M
11,1940,10,PR,M,M,M,M,M,M,M,...,0.63,1.03,0,0,T,0.18,0.58,0.5,0.25,M
12,1940,10,SN,M,M,M,M,M,M,M,...,0,0,0,0,0,0,0,0,0,0


In [9]:
weather.columns = [col for col in weather.iloc[0]]

In [10]:
weather.head()

,YR,MO,NaN,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
8,YR,MO,NaN,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
9,1940,10,TX,M,M,M,M,M,M,M,...,58,57,57,57,56,53,59,59,52,M
10,1940,10,TN,M,M,M,M,M,M,M,...,53,48,41,38,37,45,48,50,46,M
11,1940,10,PR,M,M,M,M,M,M,M,...,0.63,1.03,0,0,T,0.18,0.58,0.5,0.25,M
12,1940,10,SN,M,M,M,M,M,M,M,...,0,0,0,0,0,0,0,0,0,0


In [11]:
weather = weather[1:]

In [12]:
weather.head()

,YR,MO,NaN,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,AVG or Total
9,1940,10,TX,M,M,M,M,M,M,M,...,58,57,57,57,56,53,59,59,52,M
10,1940,10,TN,M,M,M,M,M,M,M,...,53,48,41,38,37,45,48,50,46,M
11,1940,10,PR,M,M,M,M,M,M,M,...,0.63,1.03,0,0,T,0.18,0.58,0.5,0.25,M
12,1940,10,SN,M,M,M,M,M,M,M,...,0,0,0,0,0,0,0,0,0,0
13,1940,11,TX,52,53,47,55,51,58,56,...,42,44,51,44,45,59,57,45,-,49.1


In [13]:
weather = weather.rename(columns={'YR': 'year', 'MO': 'month', np.nan: 'measure'}).drop(columns=['AVG or Total'])

In [14]:
weather.head()

,year,month,measure,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
9,1940,10,TX,M,M,M,M,M,M,M,...,61,58,57,57,57,56,53,59,59,52
10,1940,10,TN,M,M,M,M,M,M,M,...,41,53,48,41,38,37,45,48,50,46
11,1940,10,PR,M,M,M,M,M,M,M,...,0,0.63,1.03,0,0,T,0.18,0.58,0.5,0.25
12,1940,10,SN,M,M,M,M,M,M,M,...,0,0,0,0,0,0,0,0,0,0
13,1940,11,TX,52,53,47,55,51,58,56,...,44,42,44,51,44,45,59,57,45,-


In [15]:
# convert day columns to 'day', values to 'value'
weather = pd.melt(weather,
                  id_vars=['year', 'month', 'measure'],
                  value_vars=[str(day) for day in np.arange(1, 32)],
                  var_name='day',
                  value_name='value')

In [16]:
weather.head()

,year,month,measure,day,value
0,1940,10,TX,1,M
1,1940,10,TN,1,M
2,1940,10,PR,1,M
3,1940,10,SN,1,M
4,1940,11,TX,1,52


In [17]:
# convert year, month, and day values to integers for conversion to period
for col in ['year', 'month', 'day']:
    weather[col] = [int(num) for num in weather[f'{col}']]

In [18]:
# create period column
weather['period'] = [pd.Period(year=weather['year'][i], month=weather['month'][i], day=weather['day'][i], freq='D') for i in range(len(weather))]

In [19]:
# drop columns replaced by period
weather.drop(columns=['year', 'month', 'day'], inplace=True)

In [20]:
# re-order columns
weather = weather[['period', 'measure', 'value']]

In [21]:
weather.head()

,period,measure,value
0,1940-10-01,TX,M
1,1940-10-01,TN,M
2,1940-10-01,PR,M
3,1940-10-01,SN,M
4,1940-11-01,TX,52


In [22]:
demand.head()

,utc,demand_MWh
0,2015-07-22 08:00,1936.0
1,2015-07-22 09:00,1856.0
2,2015-07-22 10:00,1784.0
3,2015-07-22 11:00,1766.0
4,2015-07-22 12:00,1801.0


In [23]:
weather.sort_values(by='period', inplace=True)

In [24]:
weather.head()

,period,measure,value
0,1940-10-01,TX,M
1,1940-10-01,TN,M
2,1940-10-01,PR,M
3,1940-10-01,SN,M
3925,1940-10-02,TN,M


Values in the 'measure' column are oddly encoded.
- TX: high temp in degrees Fahrenheit
- TN: low temp in degrees Fahrenheit
- PR: precipitation in inches
- SN: snowfall, including hail and traces of hail

In [25]:
weather['measure'].value_counts()

TX    30411
TN    30411
PR    30411
SN    30411
Name: measure, dtype: int64

In [26]:
# drop precipitation and snowfall rows
weather = weather[weather['measure'] != 'PR']
weather = weather[weather['measure'] != 'SN']

In [27]:
# rename values in measure columns
weather['measure'] = ['high' if measure == 'TX' else 'low' for measure in weather['measure']]

In [28]:
# cut df to periods in demand series, those starting 7-22-15
weather = weather[weather['period'] > pd.Period(year=2015, month=7, day=21, freq='D')]

In [29]:
weather

,period,measure,value
85993,2015-07-22,low,57
85992,2015-07-22,high,72
89917,2015-07-23,low,59
89916,2015-07-23,high,80
93840,2015-07-24,high,82
...,...,...,...
113792,2022-06-29,high,NaN
117717,2022-06-30,low,NaN
117716,2022-06-30,high,NaN
121640,2022-07-01,high,NaN


In [30]:
# re-format df to have columns for high and low, period index
highs = weather[weather['measure'] == 'high'].drop(columns=['measure']).rename(columns={'value': 'high'}).set_index('period')
lows = weather[weather['measure'] == 'low'].drop(columns=['measure']).rename(columns={'value': 'low'}).set_index('period')

temps = pd.concat([highs, lows], axis=1)

In [31]:
temps.dropna(inplace=True)

In [32]:
temps

,high,low
period,,
2015-07-22,72,57
2015-07-23,80,59
2015-07-24,82,58
2015-07-25,69,60
2015-07-26,72,57
...,...,...
2022-05-20,63,46
2022-05-21,73,47
2022-05-22,76,49


In [33]:
len(temps.index.unique())

2499

In [34]:
pd.period_range(start=pd.Period('2015-07-22'), end=pd.Period('2022-05-24'))

PeriodIndex(['2015-07-22', '2015-07-23', '2015-07-24', '2015-07-25',
             '2015-07-26', '2015-07-27', '2015-07-28', '2015-07-29',
             '2015-07-30', '2015-07-31',
             ...
             '2022-05-15', '2022-05-16', '2022-05-17', '2022-05-18',
             '2022-05-19', '2022-05-20', '2022-05-21', '2022-05-22',
             '2022-05-23', '2022-05-24'],
            dtype='period[D]', length=2499)

The temps df somehow has 46 extra entries.

In [35]:
temps[temps.index.duplicated(keep=False)]

,high,low
period,,
2015-10-01,76,-
2015-10-01,-,50
2015-12-01,42,-
2015-12-01,-,34
2016-03-01,59,46
...,...,...
2022-03-02,-,-
2022-03-03,52,-
2022-03-03,-,40


In [36]:
temps[temps.index.duplicated(keep=False)][:50]

,high,low
period,,
2015-10-01,76,-
2015-10-01,-,50
2015-12-01,42,-
2015-12-01,-,34
2016-03-01,59,46
2016-03-01,-,-
2016-03-02,57,44
2016-03-02,-,-
2016-05-01,-,-


In [37]:
temps[temps.index.duplicated(keep=False)][50:]

,high,low
period,,
2019-05-01,70,-
2019-05-01,-,41
2019-07-01,-,60
2019-07-01,81,-
2019-10-01,-,37
2019-10-01,65,-
2019-12-01,43,35
2019-12-01,-,-
2020-03-01,48,34


Some dates have their high and low temps split across different entires, while other days have both temps in one entry and neither in the other.

Fortunately, the df has an entry for every date, so we just have to deal with the duplicates.

In [38]:
temps.loc[pd.Period('2022-05-01')]

,high,low
period,,
2022-05-01,-,-
2022-05-01,66,49


In [39]:
temps[temps.index.duplicated(keep=False)].index

PeriodIndex(['2015-10-01', '2015-10-01', '2015-12-01', '2015-12-01',
             '2016-03-01', '2016-03-01', '2016-03-02', '2016-03-02',
             '2016-05-01', '2016-05-01', '2016-07-01', '2016-07-01',
             '2016-10-01', '2016-10-01', '2016-12-01', '2016-12-01',
             '2017-03-01', '2017-03-01', '2017-03-02', '2017-03-02',
             '2017-03-03', '2017-03-03', '2017-05-01', '2017-05-01',
             '2017-07-01', '2017-07-01', '2017-10-01', '2017-10-01',
             '2017-12-01', '2017-12-01', '2018-03-01', '2018-03-01',
             '2018-03-02', '2018-03-02', '2018-03-03', '2018-03-03',
             '2018-05-01', '2018-05-01', '2018-07-01', '2018-07-01',
             '2018-10-01', '2018-10-01', '2018-12-01', '2018-12-01',
             '2019-03-01', '2019-03-01', '2019-03-02', '2019-03-02',
             '2019-03-03', '2019-03-03', '2019-05-01', '2019-05-01',
             '2019-07-01', '2019-07-01', '2019-10-01', '2019-10-01',
             '2019-12-01', '2019-1

In [40]:
dupes = temps[temps.index.duplicated(keep=False)]

In [41]:
dupes

,high,low
period,,
2015-10-01,76,-
2015-10-01,-,50
2015-12-01,42,-
2015-12-01,-,34
2016-03-01,59,46
...,...,...
2022-03-02,-,-
2022-03-03,52,-
2022-03-03,-,40


In [42]:
high_dupes = []
low_dupes = []

# change '-' to 0
for temp in dupes['high']:
    if temp != '-':
        high_dupes.append(temp)
    else:
        high_dupes.append(0)
        
for temp in dupes['low']:
    if temp != '-':
        low_dupes.append(temp)
    else:
        low_dupes.append(0)
        
# convert to integer
high_dupes = [int(temp) for temp in high_dupes]
low_dupes = [int(temp) for temp in low_dupes]

In [43]:
# update dupes df
dupes['high'] = high_dupes
dupes['low'] = low_dupes

C:\Users\Graham\AppData\Local\Temp\ipykernel_8296\2385596020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupes['high'] = high_dupes
C:\Users\Graham\AppData\Local\Temp\ipykernel_8296\2385596020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupes['low'] = low_dupes


In [44]:
dupes.head()

,high,low
period,,
2015-10-01,76,0
2015-10-01,0,50
2015-12-01,42,0
2015-12-01,0,34
2016-03-01,59,46


In [45]:
dupes = dupes.groupby('period').sum()

In [46]:
dupes.tail()

,high,low
period,,
2021-12-01,63,48
2022-03-01,59,50
2022-03-02,55,48
2022-03-03,52,40
2022-05-01,66,49


In [47]:
temps.head()

,high,low
period,,
2015-07-22,72,57
2015-07-23,80,59
2015-07-24,82,58
2015-07-25,69,60
2015-07-26,72,57


In [48]:
# concatenate dupes to temps, then drop rows with duplicated indices except for the last duplicate
temps = pd.concat([temps, dupes])[[not el for el in pd.concat([temps, dupes]).index.duplicated(keep='last')]]

In [49]:
temps.sort_index(inplace=True)

In [50]:
temps

,high,low
period,,
2015-07-22,72,57
2015-07-23,80,59
2015-07-24,82,58
2015-07-25,69,60
2015-07-26,72,57
...,...,...
2022-05-20,63,46
2022-05-21,73,47
2022-05-22,76,49


Temperature data clean!

### 2. Create Weekend Dummy

In [51]:
# include utc to pst adjustment (ignoring daylight saving time)
weekend = [int((period - 8).weekday in [5, 6]) for period in demand['utc']]

In [52]:
demand['weekend'] = weekend

### 3. Create Day-of-Week Dummies

In [53]:
dayofweek = [(period - 8).weekday for period in demand['utc']]

In [54]:
dow_dummies = pd.get_dummies(dayofweek, drop_first=True).rename(columns={1: 'tuesday',
                                                                         2: 'wednesday',
                                                                         3: 'thursday',
                                                                         4: 'friday',
                                                                         5: 'saturday',
                                                                         6: 'sunday'})

In [55]:
demand = pd.concat([demand, dow_dummies], axis=1)

In [56]:
demand

,utc,demand_MWh,weekend,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-07-22 08:00,1936.0,0,0,1,0,0,0,0
1,2015-07-22 09:00,1856.0,0,0,1,0,0,0,0
2,2015-07-22 10:00,1784.0,0,0,1,0,0,0,0
3,2015-07-22 11:00,1766.0,0,0,1,0,0,0,0
4,2015-07-22 12:00,1801.0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
63293,2022-10-10 13:00,1991.0,0,0,0,0,0,0,0
63294,2022-10-10 14:00,2227.0,0,0,0,0,0,0,0
63295,2022-10-10 15:00,2391.0,0,0,0,0,0,0,0
63296,2022-10-10 16:00,2414.0,0,0,0,0,0,0,0


### 4. Add Temperature Data to Demand DF

In [57]:
temps

,high,low
period,,
2015-07-22,72,57
2015-07-23,80,59
2015-07-24,82,58
2015-07-25,69,60
2015-07-26,72,57
...,...,...
2022-05-20,63,46
2022-05-21,73,47
2022-05-22,76,49


In [58]:
demand.head(25)

,utc,demand_MWh,weekend,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-07-22 08:00,1936.0,0,0,1,0,0,0,0
1,2015-07-22 09:00,1856.0,0,0,1,0,0,0,0
2,2015-07-22 10:00,1784.0,0,0,1,0,0,0,0
3,2015-07-22 11:00,1766.0,0,0,1,0,0,0,0
4,2015-07-22 12:00,1801.0,0,0,1,0,0,0,0
5,2015-07-22 13:00,1939.0,0,0,1,0,0,0,0
6,2015-07-22 14:00,2129.0,0,0,1,0,0,0,0
7,2015-07-22 15:00,2287.0,0,0,1,0,0,0,0
8,2015-07-22 16:00,2394.0,0,0,1,0,0,0,0
9,2015-07-22 17:00,2461.0,0,0,1,0,0,0,0


In [59]:
# repeat highs and lows 24 times
daily_high_spread = [[high]*24 for high in temps['high']]
daily_low_spread = [[low]*24 for low in temps['low']]

# flatten
flat_daily_high_spread = [high for l in daily_high_spread for high in l]
flat_daily_low_spread = [low for l in daily_low_spread for low in l]

In [60]:
# convert to series with hourly period index in utc, matching demand series
ser_flat_daily_high_spread = pd.Series(flat_daily_high_spread, index=pd.period_range(start='2015-07-22 08:00', end='2022-05-25 07:00', freq='H'))
ser_flat_daily_low_spread = pd.Series(flat_daily_low_spread, index=pd.period_range(start='2015-07-22 08:00', end='2022-05-25 07:00', freq='H'))

In [61]:
# update temps df
temps = pd.DataFrame({'daily_high': ser_flat_daily_high_spread, 'daily_low': ser_flat_daily_low_spread}).reset_index()

In [62]:
# rename to facilitate merge
temps.rename(columns={'index': 'utc'}, inplace=True)

In [63]:
# merge
demand = demand.merge(temps, how='left')

In [64]:
demand

,utc,demand_MWh,weekend,tuesday,wednesday,thursday,friday,saturday,sunday,daily_high,daily_low
0,2015-07-22 08:00,1936.0,0,0,1,0,0,0,0,72,57
1,2015-07-22 09:00,1856.0,0,0,1,0,0,0,0,72,57
2,2015-07-22 10:00,1784.0,0,0,1,0,0,0,0,72,57
3,2015-07-22 11:00,1766.0,0,0,1,0,0,0,0,72,57
4,2015-07-22 12:00,1801.0,0,0,1,0,0,0,0,72,57
...,...,...,...,...,...,...,...,...,...,...,...
63293,2022-10-10 13:00,1991.0,0,0,0,0,0,0,0,NaN,NaN
63294,2022-10-10 14:00,2227.0,0,0,0,0,0,0,0,NaN,NaN
63295,2022-10-10 15:00,2391.0,0,0,0,0,0,0,0,NaN,NaN
63296,2022-10-10 16:00,2414.0,0,0,0,0,0,0,0,NaN,NaN


### 5. Save Demand with Exogenous Variables

In [65]:
demand.to_csv('../data/exo_demand.csv', index=False)